In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import reduce

sys.path.append("../gaia_tools/")
import data_analysis
import transformation_constants
import covariance_generation
from import_functions import import_data

In [ ]:
path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
data_icrs = import_data(path = path, debug = False)

In [ ]:
# Transform coordinates
galcen_data = data_analysis.get_transformed_data(data_icrs,
                                        include_cylindrical = True,
                                        debug = True,
                                        is_bayes = True, 
                                        is_source_included = True)

# Generate covariance information
cov_df = covariance_generation.generate_covmatrices(df = data_icrs, 
                                        df_crt = galcen_data, 
                                        transform_to_galcen = True, 
                                        transform_to_cylindrical = True,
                                        is_bayes = True,
                                        debug=True)

galcen_data['cov_mat'] = cov_df['cov_mat']

galcen_data = galcen_data[(galcen_data.r < 12000) & (galcen_data.r > 5000) ]
galcen_data.reset_index(inplace=True, drop=True)

In [ ]:
min_val = np.min(galcen_data.r)
max_val = np.max(galcen_data.r)

# Default bins in range: -10 - 10 [kpc]
bin_collection = data_analysis.get_collapsed_bins(data = galcen_data,
                                                theta = (0, 0),
                                                BL_r_min = min_val - 1, 
                                                BL_r_max = max_val + 1, 
                                                BL_z_min = -1200, 
                                                BL_z_max = 1200, 
                                                N_bins = (10, 4),
                                                r_drift = False, 
                                                debug = True)

In [ ]:
bin_collection.GetMLEParameters()

In [ ]:
test_bin = bin_collection.bins[1]
print(test_bin.z_boundaries)
print(test_bin.data.columns)

In [ ]:
def plot_velocity_distribution(bins, is_range=False, plot_MU=False):

    if(len(bins) % np.sqrt(len(bins)) == 0):
        figs_x = round(np.sqrt(len(bins)))
        figs_y = figs_x
    else:
        figs_x = round(np.sqrt(len(bins)))
        figs_y = figs_x+1

    if(len(bins) > 5):
        figsize = 15
    else:
        figsize = 10

    fig, axs = plt.subplots(figs_y, figs_x, figsize = (figsize,figsize))

    for i, ax in enumerate(axs.flat):

        if(i < len(bins)):

            if(np.abs(np.min(bins[i].data.v_phi) - np.max(bins[i].data.v_phi)) > 500):
                n_bins = 80
            else:
                n_bins = 40

            z_range = bins[i].z_boundaries
            r_range = bins[i].r_boundaries

            text_string = "$z \in [{:.1f}, {:.1f}]$\n$r \in [{:.1f}, {:.1f}]$".format(z_range[0], z_range[1], 
                                                                                    r_range[0], r_range[1])
            ax.text(0.75, 0.8,text_string, horizontalalignment='center',
                                            verticalalignment='center',
                                            transform = ax.transAxes, fontdict={'fontsize': 12})

            if(is_range):
                mean = np.mean(bins[i].data.v_phi)
                median = np.median(bins[i].data.v_phi)
                if plot_MU: MLE_MU = bins[i].MLE_mu

                ax.hist(bins[i].data.v_phi, 
                        bins=n_bins, 
                        range = (mean-150, mean+150), 
                        edgecolor='black',
                        density = True)

                ax.axvline(x=mean, ls="--", label="Mean", color='r')
                ax.axvline(x=median, ls="--", label="Median", color='orange')
                if plot_MU: ax.axvline(x=bins[i].MLE_mu, ls="--", label="MLE", color='white')
                ax.legend(loc="lower left")

            else:    
                ax.hist(bins[i].data.v_phi, 
                        bins=n_bins, 
                        edgecolor='black')
            
            
            ax.set_xlabel("$v_\phi$ [km/s]", fontdict={'fontsize': 15}, labelpad = 5)
            
            ax.set_ylabel("N", fontdict={'fontsize': 15}, labelpad = 10, rotation=0)
            #ax.ticklabel_format(axis="y", style="sci", scilimits=(3,3))
            ax.yaxis.set_label_coords(-0.1, 1.0)

            #ax.axvline(x=0, ls="--", label="Max", color='r')
        
        else:
            fig.delaxes(ax)

    plt.tight_layout()
    #fig.suptitle("Velocity distribution in r-z bins", fontsize=18)


In [ ]:
plot_velocity_distribution(bin_collection.bins[0:4])

### BinCollection of a thinner slice  
  
  $z \in [-200, 200]$

In [ ]:
min_val = np.min(galcen_data.r)
max_val = np.max(galcen_data.r)

# Default bins in range: -10 - 10 [kpc]
bin_collection_thin = data_analysis.get_collapsed_bins(data = galcen_data,
                                                theta = (0, 0),
                                                BL_r_min = min_val - 1, 
                                                BL_r_max = max_val + 1, 
                                                BL_z_min = -200, 
                                                BL_z_max = 200, 
                                                N_bins = (10, 4),
                                                r_drift = False, 
                                                debug = True)

In [ ]:
bin_collection_thin.GetMLEParameters()

In [ ]:
for i, j  in enumerate(range(8, 28, 4)):

    plot_velocity_distribution(bin_collection_thin.bins[j:j+4], 
                                is_range=True,
                                plot_MU=False)
    plt.savefig("../out/bin_velocities/Bin_Velocities_{}.png".format(i), dpi=300)

--- 
--- 

# Variance distribution plots

In [ ]:
def plot_variance_distribution(bins, parameter, is_range=False, plot_MU=False):

    if(len(bins) % np.sqrt(len(bins)) == 0):
        figs_x = round(np.sqrt(len(bins)))
        figs_y = figs_x
    else:
        figs_x = round(np.sqrt(len(bins)))
        figs_y = figs_x+1

    if(len(bins) > 5):
        figsize = 15
    else:
        figsize = 10

    fig, axs = plt.subplots(figs_y, figs_x, figsize = (figsize,figsize))

    for i, ax in enumerate(axs.flat):

        if(i < len(bins)):

            n_bins = 160

            z_range = bins[i].z_boundaries
            r_range = bins[i].r_boundaries

            text_string = "$z \in [{:.1f}, {:.1f}]$\n$r \in [{:.1f}, {:.1f}]$".format(z_range[0], z_range[1], 
                                                                                    r_range[0], r_range[1])
            ax.text(0.75, 0.8,text_string, horizontalalignment='center',
                                            verticalalignment='center',
                                            transform = ax.transAxes, fontdict={'fontsize': 12})


            var_array = bins[i].get_error_data(parameter)
            var_array = var_array**2

            mean = np.mean(var_array)
            median = np.median(var_array)

            ax.hist(var_array, 
                    bins=n_bins,
                    #range = (0, 10*median), 
                    edgecolor='black',
                    density = True)

            ax.axvline(x=mean, ls="--", label="Mean", color='r')
            ax.axvline(x=median, ls="--", label="Median", color='orange')
            ax.legend(loc="lower right")

            ax.set_xlim(0, 0.1*np.max(var_array))

            ax.set_yscale('log')
            
            sub_index = "{" + "v_\{}".format(parameter[parameter.find('_')+1:]) + "}"
            if(parameter == 'v_r'):
                sub_index = "{" + "v_r" + "}"
            
            ax.set_xlabel('$\sigma^2_{}$ [km/s]'.format(sub_index), fontdict={'fontsize': 15}, labelpad = 5)
            
            ax.set_ylabel("N", fontdict={'fontsize': 15}, labelpad = 10, rotation=0)
            #ax.ticklabel_format(axis="y", style="sci", scilimits=(3,3))
            ax.yaxis.set_label_coords(-0.1, 1.0)
        
        else:
            fig.delaxes(ax)

    plt.tight_layout()


In [ ]:
test_arr = bin_collection_thin.bins[4].get_error_data('v_phi')
np.median((test_arr))

In [ ]:
plot_variance_distribution(bin_collection_thin.bins[4:8], parameter='v_phi',
                                plot_MU=False)

In [ ]:
plot_variance_distribution(bin_collection_thin.bins[24:28], parameter='v_r',
                                plot_MU=False)


---